In [1]:
import os
import collections
import random
import json

from data_modules.domainnet_dataset import ImageDataset
from data_modules.domainnet_metadata import DOMAIN_NET_DOMAINS, DOMAIN_NET_CLASSES, DOMAIN_NET_DIVISIONS

from torch.utils.data import DataLoader
from torchvision.transforms import v2 as T

from yasin_utils.image import imagenet_normalize

In [2]:
division_map = {}
for k, v in DOMAIN_NET_DIVISIONS.items():
    for i in v:
        division_map[i] = k

In [3]:
# root = '/data/domainnet_v1.0'
# set_map = []
# for domain in DOMAIN_NET_DOMAINS:
#     try:
#         labels = os.listdir(os.path.join(root, domain))
#     except:
#         raise Exception(f'{domain} directory not found.')
#     for label in labels:
#         for image in os.listdir(os.path.join(root, domain, label)):
#             if label in division_map:
#                 set_map.append(
#                     dict(
#                         img_path=os.path.join(root, domain, label, image),
#                         label=division_map[label],
#                         domain=domain
#                         )
#                     )

In [4]:
# random.shuffle(set_map)

In [5]:
# train_set_map, test_set_map = set_map[:int(0.9*len(set_map))], set_map[int(0.9*len(set_map)):]

In [6]:
with open('/home/yasin/repos/dispatch_smol/pretraining/data_modules/train_set_map_domainnet.json') as f:
    train_set_map_ = json.load(f)
with open('/home/yasin/repos/dispatch_smol/pretraining/data_modules/test_set_map_domainnet.json') as f:
    test_set_map_ = json.load(f)

In [7]:
train_set_map = []
for sample in train_set_map_:
    if sample['label'] in division_map:
        new_sample = {**sample}
        new_sample['label'] = division_map[sample['label']]
        train_set_map.append(new_sample)

In [8]:
test_set_map = []
for sample in test_set_map_:
    if sample['label'] in division_map:
        new_sample = {**sample}
        new_sample['label'] = division_map[sample['label']]
        test_set_map.append(new_sample)

In [9]:
len(train_set_map), len(test_set_map)

(241485, 26659)

In [10]:
dict(collections.Counter([(d['label'], d['domain']) for d in train_set_map]))

{('tool', 'quickdraw'): 12600,
 ('tool', 'painting'): 4618,
 ('electricity', 'real'): 10563,
 ('cloth', 'infograph'): 3370,
 ('tool', 'real'): 11645,
 ('mammal', 'quickdraw'): 11309,
 ('cloth', 'real'): 10397,
 ('tool', 'clipart'): 3441,
 ('mammal', 'sketch'): 4644,
 ('electricity', 'painting'): 2669,
 ('furniture', 'painting'): 4472,
 ('cloth', 'painting'): 3192,
 ('furniture', 'sketch'): 6760,
 ('building', 'quickdraw'): 9479,
 ('furniture', 'real'): 15417,
 ('building', 'real'): 10874,
 ('mammal', 'clipart'): 3096,
 ('furniture', 'quickdraw'): 15745,
 ('mammal', 'infograph'): 3212,
 ('mammal', 'real'): 13996,
 ('electricity', 'infograph'): 3744,
 ('electricity', 'quickdraw'): 11284,
 ('cloth', 'sketch'): 4374,
 ('building', 'painting'): 4610,
 ('mammal', 'painting'): 8114,
 ('electricity', 'clipart'): 2803,
 ('cloth', 'clipart'): 3275,
 ('building', 'sketch'): 4241,
 ('furniture', 'clipart'): 5199,
 ('tool', 'sketch'): 4373,
 ('cloth', 'quickdraw'): 10285,
 ('building', 'clipart'): 

In [11]:
# balance_map = {
#     'furniture': {'main_domain': 'clipart', 'target': 5_253},
#     'cloth': {'main_domain': 'real', 'target': 10_377},
#     'electricity': {'main_domain': 'infograph', 'target': 3_745},
#     'building': {'main_domain': 'painting', 'target': 4_570},
#     'mammal': {'main_domain': 'quickdraw', 'target': 11_202},
#     'tool': {'main_domain': 'sketch', 'target': 4_401},
# }

In [12]:
balance_map = {
    'furniture': {'main_domain': 'clipart', 'target': 1_000},
    'cloth': {'main_domain': 'real', 'target': 1_000},
    'electricity': {'main_domain': 'infograph', 'target': 1_000},
    'building': {'main_domain': 'painting', 'target': 1_000},
    'mammal': {'main_domain': 'quickdraw', 'target': 1_000},
    'tool': {'main_domain': 'sketch', 'target': 1_000},
}

In [13]:
# balance_map = {
#     'furniture': {'main_domain': 'clipart', 'target': 1_000},
#     'cloth': {'main_domain': 'real', 'target': 1_000},
#     'electricity': {'main_domain': 'infograph', 'target': 1_000},
#     'building': {'main_domain': 'painting', 'target': 1_000},
#     'mammal': {'main_domain': 'quickdraw', 'target': 1_000},
#     'tool': {'main_domain': 'sketch', 'target': 1_000},
# }

In [14]:
# balance_map = {
#     'furniture': {'main_domain': 'clipart', 'target': 100},
#     'cloth': {'main_domain': 'real', 'target': 100},
#     'electricity': {'main_domain': 'infograph', 'target': 100},
#     'building': {'main_domain': 'painting', 'target': 100},
#     'mammal': {'main_domain': 'quickdraw', 'target': 100},
#     'tool': {'main_domain': 'sketch', 'target': 100},
# }

In [15]:
def prune(set_map, balance_map):
    pruned_set_map = []
    
    domains = set([i['domain'] for i in set_map])
    counts = {
        l: {
            d: 0 for d in domains
        } for l in balance_map
    }
    targets = {
        l: {
            d: balance_map[l]['target'] if d == balance_map[l]['main_domain'] else int(balance_map[l]['target']*0.066) for d in domains
        } for l in balance_map
    }

    for i in set_map:
        label, domain = i['label'], i['domain']

        if counts[label][domain] < targets[label][domain]:
            pruned_set_map.append(i)

        counts[label][domain] += 1

    return pruned_set_map

In [16]:
train_set_map = prune(train_set_map, balance_map)

In [17]:
dict(collections.Counter([(d['label'], d['domain']) for d in train_set_map]))

{('tool', 'quickdraw'): 66,
 ('tool', 'painting'): 66,
 ('electricity', 'real'): 66,
 ('cloth', 'infograph'): 66,
 ('tool', 'real'): 66,
 ('mammal', 'quickdraw'): 1000,
 ('cloth', 'real'): 1000,
 ('tool', 'clipart'): 66,
 ('mammal', 'sketch'): 66,
 ('electricity', 'painting'): 66,
 ('furniture', 'painting'): 66,
 ('cloth', 'painting'): 66,
 ('furniture', 'sketch'): 66,
 ('building', 'quickdraw'): 66,
 ('furniture', 'real'): 66,
 ('building', 'real'): 66,
 ('mammal', 'clipart'): 66,
 ('furniture', 'quickdraw'): 66,
 ('mammal', 'infograph'): 66,
 ('mammal', 'real'): 66,
 ('electricity', 'infograph'): 1000,
 ('electricity', 'quickdraw'): 66,
 ('cloth', 'sketch'): 66,
 ('building', 'painting'): 1000,
 ('mammal', 'painting'): 66,
 ('electricity', 'clipart'): 66,
 ('cloth', 'clipart'): 66,
 ('building', 'sketch'): 66,
 ('furniture', 'clipart'): 1000,
 ('tool', 'sketch'): 1000,
 ('cloth', 'quickdraw'): 66,
 ('building', 'clipart'): 66,
 ('building', 'infograph'): 66,
 ('furniture', 'infograph

In [18]:
class UnbalancedDomainNetDataset(ImageDataset):
    def __init__(self, set_map, transform=None) -> None:

        super().__init__(set_map, transform)
        self.class_map = {
            'furniture': 0,
            'cloth': 1,
            'electricity': 2,
            'building': 3,
            'mammal': 4,
            'tool': 5,
        }
        self.domain_map = {
            'clipart': 0,
            'real': 1,
            'infograph': 2,
            'painting': 3,
            'quickdraw': 4,
            'sketch': 5,
        }

    def __getitem__(self, index):
        item = super().__getitem__(index)
        item['label'] = self.class_map[item['label']]
        item['domain'] = self.domain_map[item['domain']]

        return item

In [19]:
train_transform = T.Compose([
    T.RandomResizedCrop(128),
    T.RandomHorizontalFlip(),
    # T.Resize((128,128)),
    T.ToTensor(),
    imagenet_normalize,
])
val_transform = T.Compose([
    T.Resize(156),
    T.CenterCrop(128),
    # T.Resize((128,128)),
    T.ToTensor(),
    imagenet_normalize
])

/home/yasin/miniforge3/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [20]:
train_set = UnbalancedDomainNetDataset(set_map=train_set_map, transform=train_transform)
test_set = UnbalancedDomainNetDataset(set_map=test_set_map, transform=val_transform)

In [21]:
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=8, pin_memory=True, persistent_workers=True, drop_last=True)
val_loader = DataLoader(test_set, batch_size=256, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)

In [22]:
len(train_set), len(test_set)

(7980, 26659)

## Train Models

In [23]:
from typing import OrderedDict
import torch
from torch import nn
from torchvision.models.resnet import resnet50
import pytorch_lightning as L
from torchmetrics import Accuracy

In [24]:
class LinearProbe(L.LightningModule):
    def __init__(self, backbone, emb_dim, num_classes, lr=1e-3) -> None:
        super().__init__()
        
        self.backbone: nn.Module = backbone
        for param in self.backbone.parameters():
            param.requires_grad = False

        self.backbone.eval()

        # self.linear_head: nn.Module = nn.Linear(emb_dim, num_classes)
        self.linear_head: nn.Module = nn.Sequential(
            nn.Linear(emb_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.Linear(512, num_classes)
        ) 
        
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr

        self.test_accuracy = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.linear_head(x)

        return x

    def training_step(self, batch, bacth_idx):
        X = batch['image']
        t = batch['label']

        y = self.forward(X)
        loss = self.criterion(y, t)

        print(loss.item())

        return loss
    
    def test_step(self, batch, batch_idx):
        X = batch['image']
        t = batch['label']

        y = self.forward(X)
        acc = self.test_accuracy(y, t)

        self.log('accuracy', acc, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.linear_head.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 20)

        return [optimizer], [scheduler]

In [25]:
def get_backbone_from_ckpt(ckpt_path: str) -> torch.nn.Module:
    state_dict = torch.load(ckpt_path)["state_dict"]
    state_dict = OrderedDict([
        (".".join(name.split(".")[1:]), param) for name, param in state_dict.items() if name.startswith("backbone")
    ])

    return state_dict

In [26]:
# Baseline Model
model_bl  = resnet50()
weights_bl = get_backbone_from_ckpt("/home/yasin/Downloads/final_baseline.ckpt")
model_bl.load_state_dict(weights_bl, strict=False)
model_bl.fc = torch.nn.Identity()
model_bl = model_bl.cuda()

In [27]:
# MixStyle Model
model_ms  = resnet50()
weights_ms = get_backbone_from_ckpt("/home/yasin/Downloads/final_mixstyle.ckpt")
model_ms.load_state_dict(weights_ms, strict=False)
model_ms.fc = torch.nn.Identity()
model_ms = model_ms.cuda()

In [28]:
baseline_module = LinearProbe(model_bl, emb_dim=2048, num_classes=6, lr=1e-3)

In [29]:
mixstyle_module = LinearProbe(model_ms, emb_dim=2048, num_classes=6, lr=1e-3)

In [30]:
trainer = L.Trainer(
    max_epochs=20
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [31]:
trainer.fit(baseline_module, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | backbone      | ResNet             | 23.5 M
1 | linear_head   | Sequential         | 2.6 M 
2 | criterion     | CrossEntropyLoss   | 0     
3 | test_accuracy | MulticlassAccuracy | 0     
-----------------------------------------------------
2.6 M     Trainable params
23.5 M    Non-trainable params
26.1 M    Total params
104.536   Total estimated model params size (MB)
/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (31) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

1.7810397148132324
1.36961829662323
1.092119812965393
0.9771843552589417
1.1575037240982056
0.7649868130683899
0.7838195562362671
0.9604010581970215
0.7803102135658264
0.7661674618721008
0.8221171498298645
0.6671039462089539
0.7189761996269226
0.6239356398582458
0.7182053327560425
0.7173534631729126
0.6342187523841858
0.5578938126564026
0.5405734181404114
0.7385568022727966
0.6075434684753418
0.5811040997505188
0.6606559157371521
0.6409361362457275
0.4347786605358124
0.6261606216430664
0.5951897501945496
0.6176638007164001
0.5955409407615662
0.6534581780433655
0.7111263275146484
0.5203158855438232
0.42858952283859253
0.49958038330078125
0.4297848343849182
0.5063486695289612
0.4422544240951538
0.5310287475585938
0.3438812494277954
0.47810909152030945
0.37852543592453003
0.43963780999183655
0.42379385232925415
0.45459529757499695
0.4058496654033661
0.5303097367286682
0.5427607297897339
0.4561728239059448
0.4538537263870239
0.39242053031921387
0.45984452962875366
0.49451979994773865
0.415

`Trainer.fit` stopped: `max_epochs=20` reached.


In [32]:
trainer.test(baseline_module, dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 256. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9746723771095276     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.9746723771095276}]

In [33]:
trainer.test(baseline_module, dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

/home/yasin/miniforge3/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 35. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.6509246230125427     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.6509246230125427}]

In [34]:
trainer = L.Trainer(
    max_epochs=20
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [35]:
trainer.fit(mixstyle_module, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | backbone      | ResNet             | 23.5 M
1 | linear_head   | Sequential         | 2.6 M 
2 | criterion     | CrossEntropyLoss   | 0     
3 | test_accuracy | MulticlassAccuracy | 0     
-----------------------------------------------------
2.6 M     Trainable params
23.5 M    Non-trainable params
26.1 M    Total params
104.536   Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

1.7986332178115845
1.4751585721969604
1.2799625396728516
0.9796953797340393
0.9334206581115723
1.0470263957977295
0.9256604313850403
0.8343527913093567
0.8923870325088501
0.6709699630737305
0.7703192234039307
0.6284136176109314
0.6094647645950317
0.7093690037727356
0.7612224221229553
0.6368738412857056
0.7426074147224426
0.6728995442390442
0.6441876888275146
0.6590206027030945
0.6148056387901306
0.6481024622917175
0.5634718537330627
0.6407226920127869
0.6121946573257446
0.6580390334129333
0.6400423645973206
0.7136390805244446
0.7540711760520935
0.5461564064025879
0.6755139827728271
0.5839467644691467
0.43427425622940063
0.49279189109802246
0.44395923614501953
0.569694995880127
0.5148242712020874
0.6528785228729248
0.5505921244621277
0.5004514455795288
0.48739388585090637
0.510646641254425
0.43017762899398804
0.4430086016654968
0.3902919888496399
0.39890602231025696
0.4369608759880066
0.5540887713432312
0.47420623898506165
0.5352665185928345
0.47310903668403625
0.44971928000450134
0.465

`Trainer.fit` stopped: `max_epochs=20` reached.


In [36]:
trainer.test(mixstyle_module, dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9713961482048035     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.9713961482048035}]

In [37]:
trainer.test(mixstyle_module, dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.6532503366470337     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.6532503366470337}]

| # Samples | Baseline | MixStyle |
| --------- | -------- | -------- |
| All       | 0.629    | 0.642    |
| 1_000     | 0.569    | 0.576    |
| 500       | 0.520    | 0.538    |
| 100       | 0.451    | 0.463    |